# Resumir epítopos por antígeno

Dada uma lista de IDs pubmed, use o IQ-API para obter todas as informações de antígenos e epítopos
e resumir da seguinte forma:

* antígeno_id, nome, etc.
* número de referências que relatam epítopos positivos derivados desse antígeno
* num_peptídeos_positivos totais derivados de cada antígeno (cada peptídeo é contado apenas uma vez, mesmo que apareça em múltiplas referências)

fonte:  https://github.com/IEDB/IQ-API-use-cases/tree/master/python

API: https://query-api.iedb.org/docs/swagger/

https://help.iedb.org/hc/en-us/articles/4402872882189-Immune-Epitope-Database-Query-API-IQ-API

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import io
import time
import requests
import pandas as pd

In [3]:
# Read in the pubmed IDs from a file:
with open('pubmed_ids.txt') as f:
    pubmed_ids = [i.rstrip() for i in f.readlines()]
    pubmed_id_string = ','.join(pubmed_ids)

Vamos definir uma função para executar uma consulta e retornar um dataframe de resultados. Como pode haver mais de 10 mil registros, precisaremos buscar uma página por vez e continuar a anexá-los ao nosso dataframe.

**NOTA:** Ao paginar os resultados usando um 'offset', é fundamental adicionar um parâmetro 'order' para garantir que as páginas sejam consistentes entre as consultas.

Para sermos bons cidadãos, também adicionamos um breve “sono” entre as chamadas à API.

In [4]:
def iq_query(endpoint, query_params, base_uri='https://query-api.iedb.org/'):

    url = os.path.join(base_uri, endpoint)
    df = pd.DataFrame()
    
    # set the offset to 0
    query_params['offset'] = 0
    
    # loop through the pages of results
    # API only allows pulling 10,000 entries at a time
    while(True):
        print('Fetching offset: %i' % query_params['offset'])
        r = requests.get(url, params=query_params, headers={'accept': 'text/csv', 'Prefer': 'count=exact'})
        try:
            df = pd.concat([df, pd.read_csv(io.StringIO(r.content.decode('utf-8')))])
            query_params['offset'] += 10000
        except pd.errors.EmptyDataError:
            break
        
        # sleep for 1 second between calls so as not to overload the server
        time.sleep(1)
    
    return df 

### Mapear IDs publicados para IDs de referência

Vamos definir os parâmetros da string de consulta

In [5]:
query_params = {'pubmed_id': 'in.(%s)' % pubmed_id_string,
                'order': 'pubmed_id',
                'select': 'pubmed_id,reference_id'}

Extrair os dados e criar uma string de ID de referência para incorporar em consultas downstream:

In [6]:
ref2pmid = iq_query('reference_search', query_params)
ref_id_string = ','.join(list(ref2pmid['reference_id'].astype(str)))

Fetching offset: 0
Fetching offset: 10000


### Buscar os epítopos positivos das células T e B
Primeiro, definir os parâmetros de consulta:

In [7]:
query_params = {'reference_id': 'in.(%s)' % ref_id_string,
                'qualitative_measure': 'neq.Negative',
                'order': 'pubmed_id',
                'select': 'reference_id,parent_source_antigen_iri,structure_id,structure_description,curated_source_antigen'}

Agora executar a consulta por epítopos em células T:

In [8]:
tcell_epitopes = iq_query('tcell_search', query_params)

Fetching offset: 0
Fetching offset: 10000


E agora busca por epítopos em células B:

In [9]:
bcell_epitopes = iq_query('bcell_search', query_params)

Fetching offset: 0
Fetching offset: 10000


**NOTA** Há certos casos em que o antígeno de origem é desconhecido/nulo. Vamos quantificar
aqueles aqui, mas ignoraremos esses epítopos para o restante desta análise.

### Resumo inicial

Aqui resumimos o número de epítopos de células T e B recuperados e o número em que o antígeno fonte (proteína parental) não é nulo. Avançamos combinando esses dados.

In [10]:
# "~" operator is NOT; remove epitopes without a source antigen
tcell_epitopes = tcell_epitopes[~tcell_epitopes['parent_source_antigen_iri'].isna()]
bcell_epitopes = bcell_epitopes[~bcell_epitopes['parent_source_antigen_iri'].isna()]

Em ambos os casos, mais ainda nas células B, há muitos registros que não são mapeados para uma proteína parental. Investigaremos abaixo.

Vamos combinar todos os epítopos em um dataframe.

In [11]:
all_epitopes = pd.concat([tcell_epitopes, bcell_epitopes])

### Referências e peptídeos por antígeno parental

Vamos primeiro obter o número de referências e peptídeos (incluindo sequências descontínuas) por antígeno parental para cada antígeno parental.

In [12]:
ref_data  = []
peps_data = []
for i, row in all_epitopes.groupby('parent_source_antigen_iri'):
    ref_data.append([i, len(row['reference_id'].unique())])
    peps_data.append([i, len(row['structure_id'].unique())])

refs_per_parent_antigen = pd.DataFrame(data=ref_data, columns=['parent_source_antigen_iri', 'num_references'])
peps_per_parent_antigen = pd.DataFrame(data=peps_data, columns=['parent_source_antigen_iri', 'num_peptides'])


In [13]:
refs_per_parent_antigen

,parent_source_antigen_iri,num_references
0,UNIPROT:A0A0U1RQC9,1
1,UNIPROT:A2JGV3,2
2,UNIPROT:A5PLL7,3
3,UNIPROT:B4DTR1,1
4,UNIPROT:D6RHG4,1
5,UNIPROT:H0YKS8,1
6,UNIPROT:H3BLT4,1
7,UNIPROT:I3L0A0,1
8,UNIPROT:O14746,1
9,UNIPROT:O15438,2


### Dados de antígeno de origem (proteína mãe)

Antes de juntarmos tudo, precisamos extrair todas as informações do antígeno de origem. Como parent_source_antigen_iri ainda não está na tabela parent_proteins, primeiro extraímos TODAS as proteínas-mãe
e crie parent_source_antigen_uri mais tarde.

In [14]:
query_params = {'order': 'accession',
                'select': 'database,accession,name,title,proteome_label'}

Agora executar a consulta em parent_proteins para extrair tudo

In [15]:
parent_proteins = iq_query('parent_proteins', query_params)

Fetching offset: 0
Fetching offset: 10000
Fetching offset: 20000
Fetching offset: 30000
Fetching offset: 40000
Fetching offset: 50000
Fetching offset: 60000
Fetching offset: 70000
Fetching offset: 80000
Fetching offset: 90000
Fetching offset: 100000
Fetching offset: 110000


In [16]:
# vamos adicionar um pseudo parent_source_antigen_iri
parent_proteins['parent_source_antigen_iri'] = parent_proteins['database'].str.upper() + ':' + parent_proteins['accession'].str[:]

### Resumo final por proteína pai

Aqui, combinamos os dados na tabela de interesse

In [19]:
final_summary_by_parent = pd.merge(refs_per_parent_antigen, peps_per_parent_antigen, how='left', on='parent_source_antigen_iri')
final_summary_by_parent = pd.merge(final_summary_by_parent, parent_proteins, how='left', on='parent_source_antigen_iri')

# renomear colunas
final_summary_by_parent.rename(columns={'parent_source_antigen_iri': 'antigen_id',
                                        'title': 'antigen_title'}, inplace=True)

# ordenar pelo número de referências e número de peptídeos
final_summary_by_parent = final_summary_by_parent.sort_values('num_references', ascending=False)
final_summary_by_parent = final_summary_by_parent.sort_values('num_peptides', ascending=False)

final_summary_by_parent

,antigen_id,num_references,num_peptides,database,accession,name,antigen_title,proteome_label
18,UNIPROT:P07288,5,8,UniProt,P07288,sp|P07288|KLK3_HUMAN,Prostate-specific antigen,Homo sapiens (Human)
22,UNIPROT:P12272,2,8,UniProt,P12272,sp|P12272|PTHR_HUMAN,Parathyroid hormone-related protein,Homo sapiens (Human)
37,UNIPROT:Q15020,4,8,UniProt,Q15020,sp|Q15020|SART3_HUMAN,Squamous cell carcinoma antigen recognized by ...,Homo sapiens (Human)
16,UNIPROT:P06239,3,7,UniProt,P06239,sp|P06239|LCK_HUMAN,Tyrosine-protein kinase Lck,Homo sapiens (Human)
26,UNIPROT:P22732,1,7,UniProt,P22732,sp|P22732|GTR5_HUMAN,"Solute carrier family 2, facilitated glucose t...",Homo sapiens (Human)
32,UNIPROT:P63145,1,6,UniProt,P63145,sp|P63145|GAK24_HUMAN,Endogenous retrovirus group K member 24 Gag po...,Homo sapiens (Human)
31,UNIPROT:P62684,1,6,UniProt,P62684,sp|P62684|GA113_HUMAN,Endogenous retrovirus group K member 113 Gag p...,Homo sapiens (Human)
35,UNIPROT:Q04609,3,5,UniProt,Q04609,sp|Q04609|FOLH1_HUMAN,Glutamate carboxypeptidase 2,Homo sapiens (Human)
30,UNIPROT:P60484,1,5,UniProt,P60484,sp|P60484|PTEN_HUMAN,"Phosphatidylinositol 3,4,5-trisphosphate 3-pho...",Homo sapiens (Human)
24,UNIPROT:P15309,3,4,UniProt,P15309,sp|P15309|PPAP_HUMAN,Prostatic acid phosphatase,Homo sapiens (Human)


Como podemos ver pela linha com antígeno_id e antígeno_title vazios, há um bom número de proteínas que não estão mapeadas para uma proteína mãe. Então, vamos dar uma olhada nisso, especificamente:

In [18]:
all_epitopes['curated_antigen_accession'] = all_epitopes['curated_source_antigen'].str.split(',').str[0].str[1:]
all_epitopes[all_epitopes['parent_source_antigen_iri'].isna()]

,reference_id,parent_source_antigen_iri,structure_id,structure_description,curated_source_antigen,curated_antigen_accession


Interessante! Nenhum deles possui antígenos de origem selecionados, então devemos investigar o porquê e ignorá-los se houver uma boa explicação.